In [ ]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 18.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
!pip install jdc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.multiprocessing
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

GLOBAL_SEED = 42  # number of life
torch.manual_seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
total_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ĐA1/Comments.csv', header=None)
course_info = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ĐA1/Course_info.csv', header=None)

<ipython-input-5-f823554466ca>:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  total_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ĐA1/Comments.csv', header=None)
<ipython-input-5-f823554466ca>:2: DtypeWarning: Columns (0,2,3,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  course_info = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ĐA1/Course_info.csv', header=None)


In [ ]:
total_data.drop(index=0, inplace=True)
total_data.rename(columns={0: 'comment_id', 1: 'course_id', 2: 'rating', 3: 'date', 4: 'user_name', 5: 'comment'}, inplace=True)
total_data

,comment_id,course_id,rating,date,user_name,comment
1,88962892,3173036,1.0,2021-06-29T18:54:25-07:00,Rahul,I think a beginner needs more than you think.\...
2,125535470,4913148,5.0,2022-10-07T11:17:41-07:00,Marlo,Aviva is such a natural teacher and healer/hea...
3,68767147,3178386,3.5,2020-10-19T06:35:37-07:00,Yamila Andrea,Muy buena la introducción para entender la bas...
4,125029758,3175814,5.0,2022-09-30T21:13:49-07:00,Jacqueline,This course is the best on Udemy. This breakd...
5,76584052,3174896,4.5,2021-01-30T08:45:11-08:00,Anthony,I found this course very helpful. It was full ...
...,...,...,...,...,...,...
9411723,54471142,567828,5.0,2020-05-10T08:06:29-07:00,Rogelio,I waited to completely finish the course to ra...
9411724,54469534,567828,5.0,2020-05-10T07:51:34-07:00,Rao Saurabh,it was amazing. i learnt a lot
9411725,54466660,567828,5.0,2020-05-10T07:22:30-07:00,Raveesh,Just started with the course and so far the pa...
9411726,54462674,567828,5.0,2020-05-10T06:42:43-07:00,Wendell,"So far, this course has been a really good and..."


In [ ]:
data = total_data[0:500000]

In [ ]:
users = data['user_name'].unique()
users = pd.DataFrame(users, columns=['user_name'])
users['user_id'] = range(1, len(users) + 1)

In [ ]:
courses = data['course_id'].unique()
courses = pd.DataFrame(courses, columns=['course_id'])

In [ ]:
ratings = data[['course_id', 'rating', 'user_name']]

In [ ]:
ratings['rating'] = ratings['rating'].astype(float)
ratings["rating"] = ratings["rating"] - 3  # rating range (-2, 2)
train_ratings, validation_ratings = train_test_split(
    ratings, test_size=0.1, random_state=GLOBAL_SEED
)

<ipython-input-53-53c0b39177f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['rating'] = ratings['rating'].astype(float)
<ipython-input-53-53c0b39177f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings["rating"] = ratings["rating"] - 3  # rating range (-2, 2)


In [ ]:
course_index_by_id_bonus = {id: idx for idx, id in enumerate(course_info[0])}
del course_index_by_id_bonus['id']

In [ ]:
course_index_by_id_bonus = {float(key): value for key, value in course_index_by_id_bonus.items()}
course_index_by_id_bonus = {int(key): value for key, value in course_index_by_id_bonus.items()}
course_index_by_id_bonus = {str(key): value for key, value in course_index_by_id_bonus.items()}
course_index_by_id_bonus

{'4715': 1,
 '1769': 2,
 '5664': 3,
 '7723': 4,
 '8157': 5,
 '8139': 6,
 '2762': 7,
 '8082': 8,
 '8075': 9,
 '8069': 10,
 '8318': 11,
 '8422': 12,
 '8416': 13,
 '8420': 14,
 '8324': 15,
 '8467': 16,
 '8471': 17,
 '8319': 18,
 '8410': 19,
 '8629': 20,
 '8776': 21,
 '8814': 22,
 '8746': 23,
 '8621': 24,
 '8812': 25,
 '9071': 26,
 '8874': 27,
 '9061': 28,
 '8849': 29,
 '9287': 30,
 '9224': 31,
 '9327': 32,
 '9236': 33,
 '9385': 34,
 '9646': 35,
 '9521': 36,
 '9513': 37,
 '9463': 38,
 '9707': 39,
 '9712': 40,
 '10413': 41,
 '9711': 42,
 '10251': 43,
 '10318': 44,
 '10620': 45,
 '10320': 46,
 '10317': 47,
 '10321': 48,
 '10449': 49,
 '10723': 50,
 '10967': 51,
 '10629': 52,
 '10681': 53,
 '10971': 54,
 '10935': 55,
 '10973': 56,
 '10722': 57,
 '10812': 58,
 '10964': 59,
 '11152': 60,
 '11331': 61,
 '11197': 62,
 '11305': 63,
 '11051': 64,
 '11418': 65,
 '11153': 66,
 '11670': 67,
 '11475': 68,
 '11922': 69,
 '12123': 70,
 '12226': 71,
 '11967': 72,
 '12137': 73,
 '11777': 74,
 '12196': 75,


In [ ]:
topics = []
for id in data['course_id']:
    try:
      topics.append(course_info[13][course_index_by_id_bonus[id]])
    except:
      topics.append('None')
data['topic'] = topics

<ipython-input-48-45686aa6a268>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['topic'] = topics


In [ ]:
data['topic'].nunique()

14

In [ ]:
# map course id and user name to indexes.
course_index_by_id = {id: idx for idx, id in enumerate(courses["course_id"])}
user_index_by_name = {id: idx for idx, id in enumerate(users["user_name"])}

In [ ]:
class MLDataset(Dataset):
    def __init__(self, ratings: pd.DataFrame):
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, index):
        user_name = self.ratings["user_name"].iloc[index]
        course_id = self.ratings["course_id"].iloc[index]
        rating = self.ratings["rating"].iloc[index]
        user_index = user_index_by_name[user_name]
        course_index = course_index_by_id[course_id]
        return user_index, course_index, rating


training_data = MLDataset(train_ratings)
validation_data = MLDataset(validation_ratings)
batch_size = 1024
train_dataloader = DataLoader(
    training_data, batch_size=batch_size, shuffle=True, num_workers=10
)
validation_dataloader = DataLoader(
    validation_data, batch_size=batch_size, shuffle=False, num_workers=10
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import jdc

LR = 1 #learning rate
WEIGHT_DECAY = 5e-5 #Siêu tham số alpha trong regularization - thường rất nhỏ


class MatrixFactorization(pl.LightningModule):
    """Pytorch lighting class for Matrix Factorization training.

    Attributes:
        n_users: number of users.
        n_items: number of items.
        n_factors: number of latent factors (or embedding size)
    """

    def __init__(self, n_users: int, n_items: int, n_factors: int = 14):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors
        self.user_biases = nn.Embedding(n_users, 1)
        self.item_biases = nn.Embedding(n_items, 1)
        self.bias = nn.Parameter(data=torch.rand(1))
        self.user_embeddings = nn.Embedding(n_users, n_factors)
        self.item_embeddings = nn.Embedding(n_items, n_factors)

    def forward(self, users, items):
        """
        Forward pass through the model. For a single user and item, this
        looks like:
        bias + user_bias + item_bias + user_embeddings.dot(item_embeddings)

        Arguments:
            users: Array of user indices
            items : Array of item indices
        Returns:
            preds: Predicted ratings.
        """
        # select users and items from the batch
        batch_user_embs = self.user_embeddings(users)
        batch_item_embs = self.item_embeddings(items)

        preds = torch.reshape(
            torch.diag(
                torch.matmul(batch_user_embs, torch.transpose(batch_item_embs, 0, 1))
            ),
            (-1, 1),
        )
        # add bias
        preds += self.user_biases(users) + self.item_biases(items) + self.bias

        return torch.clip(preds.squeeze(), min=-2, max=2)

    def training_step(self, batch, batch_idx):
        users, items, rating = batch
        rating = rating.to(torch.float32)
        output = self.forward(users, items)
        loss = F.mse_loss(rating, output)
        self.log("batch_loss", loss)
        return {"loss": loss}  # for computing avg_loss in training_epoch_end

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        return optimizer

In [ ]:
# for tensorboard
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger("mf_tb_logs", name=f"lr{LR}_wd{WEIGHT_DECAY}")

n_users = len(user_index_by_name)
n_movies = len(course_index_by_id)
n_factors = 14
model = MatrixFactorization(n_users=n_users, n_items=n_movies, n_factors=n_factors)
trainer = pl.Trainer(max_epochs=100, logger=logger)
trainer.fit(model, train_dataloader, validation_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type      | Params
----------------------------------------------
0 | user_biases     | Embedding | 147 K 
1 | item_biases     | Embedding | 97.5 K
2 | user_embeddings | Embedding | 2.1 M 
3 | item_embeddings | Embedding | 1.4 M 
  | other params    | n/a       | 1     
----------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.712    Total estimated model 

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
#14 fators - 500k data
def eval_model(model, train_dataloader):
    loss = 0
    for users, items, rating in train_dataloader:
        pred = model(users, items)
        loss += F.mse_loss(pred, rating)
    RMSE = (loss / len(train_dataloader))**.5
    return RMSE
print("Train RMSE: {:.3f}".format(eval_model(model, train_dataloader)))
print("Validation RMSE: {:.3f}".format(eval_model(model, validation_dataloader)))

Train RMSE: 0.904
Validation RMSE: 1.042
